In [3]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.datasets import load_iris
import numpy as np

iris = load_iris(as_frame=True)
data = iris.data[['sepal length (cm)', 'sepal width (cm)']].rename(columns={'sepal length (cm)': 'sepal_length', 'sepal width (cm)': 'sepal_width'})
target = iris.target
data

,sepal_length,sepal_width
0,5.1,3.5
1,4.9,3.0
2,4.7,3.2
3,4.6,3.1
4,5.0,3.6
...,...,...
145,6.7,3.0
146,6.3,2.5
147,6.5,3.0
148,6.2,3.4


In [4]:
from sklearn.model_selection import train_test_split

X = data[0:100]
y = target[0:100]

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.25)

In [18]:
X

,sepal_length,sepal_width
0,5.1,3.5
1,4.9,3.0
2,4.7,3.2
3,4.6,3.1
4,5.0,3.6
...,...,...
95,5.7,3.0
96,5.7,2.9
97,6.2,2.9
98,5.1,2.5


In [16]:
from sklearn.naive_bayes import GaussianNB

gnb = GaussianNB()
y_pred = gnb.fit(X_train, y_train).predict(X_test)

mean_per_class = gnb.theta_
var_per_class = gnb.var_

array([[4.96363636, 3.37272727],
       [5.91428571, 2.77619048]])

In [14]:
gnb_coefs = [list(np.append(np.append(x,y),z)) for x, y, z in zip(['Class1','Class2'], mean_per_class, var_per_class)]
gnb_coefs

[['Class1',
  '4.963636363636364',
  '3.3727272727272726',
  '0.14231405002476086',
  '0.17955922908812175'],
 ['Class2',
  '5.914285714285713',
  '2.7761904761904765',
  '0.2726530616624738',
  '0.10086167844251918']]

In [9]:
import os

import pyodbc
import numpy as np
import matplotlib.pyplot as plt
from dotenv import load_dotenv

load_dotenv()
db_username = os.getenv('DB_USERNAME')
db_password = os.getenv('DB_PASSWORD')

server = 'DESKTOP-E2GMA60'
database = 'Iris_Classif'
username = db_username
password = db_password

In [17]:
cnxn = pyodbc.connect('DRIVER={SQL Server};SERVER='+server+';DATABASE='+database+';UID='+username+';PWD='+password)
cursor = cnxn.cursor()
cursor.executemany("INSERT INTO data (class, [length mean], [width mean], [length variance], [width variance]) values (?, ?, ?, ?, ?)", gnb_coefs)
cnxn.commit()
cursor.close()
cnxn.close()

In [ ]:
gnb_coefs = []
log_prob = []
for i in range(len(X_test)):
    log_prob_i = []
    for j in range(len(mean_per_class)):
        log_prob_i.append(np.sum(np.log(np.exp(-0.5 * ((X_test[i] - mean_per_class[j])**2 / var_per_class[j])) / np.sqrt(2 * np.pi * var_per_class[j]))))
    log_prob.append(log_prob_i)